<a href="https://colab.research.google.com/github/GuntawitBen/AI-Study-Assistant-with-Gemini-and-Drive-Ingestion/blob/main/EGCI461_AI_FinalProject_StudyBuddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**EGCI461 - Artificial Intelligence**

**Final Project - EGCI461 Study Buddy**

**Members**
1.   Guntawit Anurakboonying 6481260
2.   Pitchapa Phisupichet 6580065
3.   Natnicha Sukchuenanant 6580212





In [2]:
# Project Requirements

# The chatbot should be able to:
#     Understand and respond to user inputs
#     Provide meaningful, relevant responses
#     Be implemented in Python (you may use libraries such as nltk, transformers, ChatterBot)

# Submit their source code
# Provide a short report (1–2 pages)
# Prepare a 5–7 minute presentation and demo of the chatbot

In [3]:
# INSTALL DEPENDENCIES

!pip install PyMuPDF
#!pip install transformers
!pip -q install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 68.1 MB/s eta 0:00:00


In [21]:
import google.generativeai as genai

from google.colab import userdata
from IPython.display import Markdown

GOOGLE_AI_STUDIO = userdata.get('Gemini')

genai.configure(api_key=GOOGLE_AI_STUDIO)

In [5]:
for m in genai.list_models():
  print(m.name)

models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-ex

In [25]:
generation_config = {
    "temperature": 0.3,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 300,
}

safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

model = genai.GenerativeModel(
    model_name="gemini-2.0-flash-lite",
    generation_config=generation_config,
    safety_settings=safety_settings,
)

In [29]:
# UPLOAD PDF FILES

from google.colab import drive
drive.mount('/content/drive')

import os
import fitz

def extract_text_from_folder(folder_path):
    combined_text = ""
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            doc = fitz.open(file_path)
            for page in doc:
                combined_text += page.get_text()
    return combined_text

midterm_path = "/content/drive/MyDrive/EGCI461/midterm"
final_path = "/content/drive/MyDrive/EGCI461/final"

midterm_text = extract_text_from_folder(midterm_path)
final_text = extract_text_from_folder(final_path)

print("Extracted text from all PDFs in Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracted text from all PDFs in Drive.


In [8]:
word_count = len(midterm_text.split())
print(f"The number of words in midterm_text is: {word_count}")
word_count = len(final_text.split())
print(f"The number of words in final_text is: {word_count}")

# midterm has about 80,000 tokens

The number of words in midterm_text is: 16069
The number of words in final_text is: 9330


In [26]:
prompt = "What is ai in 1 sentence?"
response = model.generate_content([prompt])
print("Bot:", response.text)

Bot: AI, or artificial intelligence, is the simulation of human intelligence processes by computer systems, enabling them to perform tasks that typically require human intelligence.



In [13]:
def format_response_text(text, words_per_line=20, indent='     '):
    words = text.split()
    lines = [' '.join(words[i:i+words_per_line]) for i in range(0, len(words), words_per_line)]
    # Add indent to all lines except the first
    lines = [lines[0]] + [indent + line for line in lines[1:]]
    return '\n'.join(lines)

In [67]:
def handle_ask(context):
    print("\nBot: Alright! You can ask questions now. You can always type 'quit' to return to the main menu.")
    chat = model.start_chat(history=[])
    while True:
        question = input("You: ").strip()
        if question.lower() == "quit" or question.lower() == "bye":
            print("Bot: Exiting Q&A mode.\n")
            break

        print("Bot: Thinking...")
        prompt = (
            f"You are a study assistant. Answer based on the following course notes. "
            f"If content is not found in the notes, use external sources. "
            f"\n\nAnswer clearly and concisely in 1-2 sentences."
            f"\n\nCourse Notes:\n{context}\n\n"
            f"Question: {question}\n"
            f"Answer:"
        )

        response = chat.send_message(prompt)
        formatted_text = format_response_text(response.text, words_per_line=20)
        print(f"Bot: {formatted_text}")

In [68]:
import re

def handle_quiz(context):
    chat = model.start_chat(history=[])

    while True:
        print("\nBot: Alright! Press 'Enter' when you are ready. Type 'quit' anytime to return to the main menu.")
        question = input("You: ").strip()
        if question.lower() == "quit" or question.lower() == "bye":
            print("Bot: Exiting Quiz mode.\n")
            break


        # 1. Generate Question =========================================================================================
        generate_prompt = (
            f"You are a study assistant. Based on the following course notes, generate exactly 1 short quiz question\n"
            f"answer with its concise correct answer for student revision. Format as:\n"
            f"Answer: ...\n\n"
            f"Question: ...\n"
            f"Course Notes:\n{context}\n\n"
            f"Quiz:"
        )
        response = chat.send_message(generate_prompt)

        text = response.text.strip()

        question_match = re.search(r"Question:(.*?)(?:Answer:|$)", text, re.DOTALL | re.IGNORECASE)
        answer_match = re.search(r"Answer:(.*)", text, re.DOTALL | re.IGNORECASE)

        if question_match and answer_match:
            question_text = question_match.group(1).strip()
            correct_answer = answer_match.group(1).strip()
        else:
            print("Bot: Could not parse a valid quiz question. Retrying...")
            continue

        formatted_question = format_response_text(question_text, words_per_line=20)
        print(f"Bot: {formatted_question}")

        # 2️. Get student answer =======================================================================================================================
        student_answer = input("You: ").strip()
        if student_answer.lower() in ["quit", "bye"]:
            print("Bot: Exiting Quiz mode.\n")
            break

        # 3️. Prompt Gemini to check answer ============================================================================================================
        check_prompt = (
            f"You are a study assistant helping a student. Determine if the student's answer is correct comparing to the correct answer for the quiz question below.\n"
            f"Check the Student's Answer: only\n"
            f"Respond with only 'Correct' or 'Incorrect'. Do not add anything else.\n\n"
            f"Question: {question_text}\n"
            f"Correct Answer: {correct_answer}\n"
            f"Student's Answer: {student_answer}"
        )
        check_response = chat.send_message(check_prompt)
        check_result = check_response.text.strip().lower()

        if check_result == "correct":
          print("Bot: Correct! Great job!")
        else:
            # 4️. Explain why the correct answer is correct ============================================================================================
            explain_prompt = (
                f"The student answered the following quiz question incorrectly. Provide a friendly, concise explanation (2-3 sentences) "
                f"clarifying why the correct answer is correct, helping the student understand the concept.\n\n"
                f"Question: {question_text}\n"
                f"Correct Answer: {correct_answer}\n"
                f"Student's Answer: {student_answer}"
            )
            explanation_response = chat.send_message(explain_prompt)
            formatted_explanation = format_response_text(explanation_response.text.strip(), words_per_line=20)

            print(f"Bot: {formatted_explanation}")


In [54]:
# RULE BASED CHATBOT FOR MENU

def chatbot():
  print("EGCI461 Study Buddy Bot")
  print("Type 'bye' anytime to exit.\n")
  print("Bot: Hello! Ready to study? You can tell me if you wanna study for 'midterm' or 'final!")

  topic = None
  mode = None

  while True:


      user_input = input("You: ").strip().lower()

      if "bye" in user_input:
          print("Bot: Good luck on your exam!")
          break

      elif "hello" in user_input or "hi" in user_input or "hey" in user_input:
          print("Bot: Hi there! Do wanna study for 'midterm' or 'final'?")

      elif "how are you" in user_input or "what you do" in user_input:
          print("Bot: I'm here to help you study AI course!")

      elif "name" in user_input:
          print("Bot: I am Study Buddy Bot!")

      elif "midterm" in user_input:
          topic = "midterm"
          if mode == "ask":
            handle_ask(midterm_text)
          elif mode == "quiz":
            handle_quiz(midterm_text)
          else:
            print("Bot: We will study for Midterm! Do you want to 'ask' questions or take a 'quiz'?")

      elif "final" in user_input:
          topic = "final"
          if mode == "ask":
            handle_ask(final_text)
          elif mode == "quiz":
            handle_quiz(final_text)
          else:
            print("Bot: We will study for Final! Do you want to 'ask' questions or take a 'quiz'?")

      elif "ask" in user_input:
          if topic == "midterm":
              handle_ask(midterm_text)
          elif topic == "final":
              handle_ask(final_text)
          else:
              mode = "ask"
              print("Bot: Sure! Do you want to ask about 'midterm' or 'final'?")


      elif "quiz" in user_input:
          if topic == "midterm":
              handle_quiz(midterm_text)
          elif topic == "final":
              handle_quiz(final_text)
          else:
              mode = "quiz"
              print("Bot: Sure! Do you want to take a quiz about 'midterm' or 'final'?")


      else:
          print("Bot: I didn't understand that. Do you want to study for 'midterm' or 'final'?")


In [66]:
chatbot()

EGCI461 Study Buddy Bot
Type 'bye' anytime to exit.

Bot: Hello! Ready to study? You can tell me if you wanna study for 'midterm' or 'final!
You: midterm
Bot: We will study for Midterm! Do you want to 'ask' questions or take a 'quiz'?
You: ask

Alright! You can ask questions now. You can always type 'quit' to return to the main menu.

You: what is network address
Bot: Thinking...
Bot: I am sorry, I cannot answer that question. I am not able to provide information on networking topics.
You: what should i eat today
Bot: Thinking...
Bot: I am sorry, I cannot answer that question. I am not able to provide information on food or dietary topics.
You: bye
Bot: Exiting Q&A mode.
You: bye
Bot: Good luck on your exam!
